In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
from sentence_transformers import SentenceTransformer
emb_mod = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

load_dotenv("../.env")
openAI_api_key = os.environ.get("OPENAI_API_KEY")

/home/pwnphofun/miniconda3/envs/codenames/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
client = OpenAI(api_key=openAI_api_key)

In [22]:
team = "blue"
enemy = "red"
system_prompt = f"""

You are a human that plays the game Codenames utilising common knowledge.
**Your Task: You will be provided a list of {team} target words, {enemy} enemy words, neutral civilian words, and an Assassin word. Your goal is to generate a list of high-quality and **immediately understandable** clues, each targeting as many of the {team} target words as possible while maximizing clarity and minimizing ambiguity. Each clue also has to absolutely avoid any connnection to the assassin word. You might provide up to 5 clues.**

**Clue Generation Principles:**
1. Clue Rules (Crucial):
    * **Strong Semantic Relation:** Must be **immediately and clearly** semantically related to **as many of the target words as possible**, using *primary meanings* or *widely recognized themes* that a typical human guesser would quickly understand.
    * **Single Word:** Must be a single English word.
    * **No Direct Forms:** Must NOT be a form (inflection, derivation) of any word currently visible on the board (e.g., if 'DRIVE' is on board, 'DRIVER', 'DRIVING', 'DRIVES' are invalid).
    * **No Substring/Superstring Containment (if related):** Must NOT contain a word on the board (e.g., if 'FIRE' is on board, 'FIREMAN' is invalid) or be a substring of another word on the board (e.g., if 'SKYLINE' is on the board, 'SKY' is invalid).

2. **Overall Clue Quality (Reinforcing Clarity):**
    * **Strength & Directness of Association:** The clue must strongly connect to *as many of the target words as possible** via their *primary meanings* or *obvious shared themes* (e.g., “fruit” for APPLE, BANANA, ORANGE is clear).
    * **Unambiguity:** Ensure the clue has minimal overlap with the Assassin word. This is the most critical negative constraint.
    * **Minimal Connection to the {enemy} enemy words and civilian words**: After minimizing the overlap with the Assassin word, minize your clues' associations with the enemy words and civilian words. A bad clue will make your teammate guesser guesses the enemy word and give the enemy an advantage. 
    * **Think Like a Typical Human Guesser:** The clue must be instantly recognizable and understandable to a typical human with general knowledge (not an AI or expert). Avoid clues that rely on niche, context-specific, or obscure associations.
    * **Avoid clue words that need to pair with other words to make a phrase**: for example, the clue "hair" that targets "FALL" and "CYCLE" is unacceptable if your reasoning is it's supposed to evoke “hair fall” and “hair cycle". 
    * **High-Quality Clues**: high-quality clues represent complex linguistic relationships such as syno-, anto-, hyper-, hypo- and meronymy. 
    
3. **Safety First (Assassin Avoidance):**
    * **ABSOLUTELY AVOID** any clue that has even a remote chance of relating to any concepts related to he ASSASSIN word. This is the highest priority negative constraint.

* **Examples of Good vs. Bad Clues:**
        * **Good Clue:** “Ocean” for WAVE, TIDE, CORAL (clear, direct, universally understood theme of ocean-related terms).
        * **Bad Clue:** “Hair” for BRUSH, FALL, CYCLE (niche associations like “hair fall” and “hair cycle” are not obvious; a human might think of SCALP or COMB instead).
        * **Bad Clue:** “Revolver” for PISTOL, PLOT, WITCH (connection to PLOT is abstract, and WITCH is unrelated for humans).
        * **Bad Clue:** “djedkare” is a bad clue. Even though it refers to the name of the ruler of Egypt in the 25thcentury B.C., and therefore connects the words “egypt” and “king.", it is so niche that it does not reflect the average person’s knowledge of the English language and is likely to yield random guesses if presented to a human player. 


You may choose up to 5 clues. For each clue, think about why you would choose it, and revisit the guidelines/strategies given to see if the clue matches all the criterias. If you picked a clue and later realized that it's not good, then discard that clue. For each clue, provide reasoning that includes:
* why it relates to the provided list of target words. 
* why it adheres to the guidelines/rules provided.
* Confirmation that the clue avoids the Assassin word. 

**Output Format:**
On a new line at the end of your response, list the clues separated by commas, in order starting with the best clue. All clues must be on the same line at the end.
EXAMPLE if you choose to provide 3 clues: 
clue1, clue2, clue3
"""

In [23]:
history = [{"role": "system", "content": system_prompt}]

In [24]:
red_words = [
    "amazon", "hole", "shark", "ground", "shop", "cast", "nurse", "server", "vacuum", "rock"
]
red_words = [word.upper() for word in red_words]

blue_words = [
    "crown", "pit", "change", "glove", "charge", "torch", "whip", "fly", "africa", "giant"
]
blue_words = [word.upper() for word in blue_words]

neutral_words = [
    "CONDUCTOR", "HEART", "TOKYO",
    "BOW", "DANCE", "LIFE", "SOCK"
]

assassin_word = "AGENT"


In [25]:
prompt = ""
prompt += "The remaining words are: "
prompt += "Red: " + ", ".join(red_words) + ".\n"
prompt += "Blue: " + ", ".join(blue_words) + ".\n"
prompt += "Civilian: " + ", ".join(neutral_words) + ".\n"
prompt += "Assassin: " + assassin_word + ". "
prompt += f"**You are the {team} Codemaster. Analyze the provided game state and generate your list of candidate clues now. Remember that your clues must adhere to the rules provided in the Game Overview section.**"
print(prompt)
history.append({"role": "user", "content": prompt})


The remaining words are: Red: AMAZON, HOLE, SHARK, GROUND, SHOP, CAST, NURSE, SERVER, VACUUM, ROCK.
Blue: CROWN, PIT, CHANGE, GLOVE, CHARGE, TORCH, WHIP, FLY, AFRICA, GIANT.
Civilian: CONDUCTOR, HEART, TOKYO, BOW, DANCE, LIFE, SOCK.
Assassin: AGENT. **You are the blue Codemaster. Analyze the provided game state and generate your list of candidate clues now. Remember that your clues must adhere to the rules provided in the Game Overview section.**


In [26]:
start = time.time()
response = client.chat.completions.create(
            messages=history,
            model="gpt-4.1",
        ).choices[0].message.content

print(f"Time to generate clues: {time.time() - start}")

Time to generate clues: 30.636608600616455


In [27]:
print(response)

Let’s first analyze the word groups:

**Blue targets:**  
CROWN, PIT, CHANGE, GLOVE, CHARGE, TORCH, WHIP, FLY, AFRICA, GIANT

**Red (danger words):**  
AMAZON, HOLE, SHARK, GROUND, SHOP, CAST, NURSE, SERVER, VACUUM, ROCK

**Civilians:**  
CONDUCTOR, HEART, TOKYO, BOW, DANCE, LIFE, SOCK

**Assassin:**  
AGENT

## Step 1: Cluster blue targets into possible clue groupings

Let’s look for groups that can be safely linked with a single concept:

### A. CROWN, GIANT, AFRICA
- “Africa” is unique and not otherwise paired. “Giant” and “Crown” can sometimes be linked in myth/royalty, but hard to tie with "AFRICA" safely.

### B. GLOVE, WHIP, TORCH
- “Glove” and “Whip” could fit with "TORCH" as equipment/tools or something a person holds.
- However, “GLOVE” and “WHIP” can also relate to sports/boxing, "TORCH" to Olympics.

### C. CHANGE, CHARGE
- Both relate to money or electrical concepts. "CONDUCTOR" is a civilian, so "electric" clues are risky.

### D. FLY, PIT
- Not much overlap unless going 

In [49]:
print(response)

I’ll explain my reasoning for the chosen clue:

Chosen clue: Mummy

• Connection to GHOST: A mummy is a classic undead, ghostly figure in horror stories.  
• Connection to COMIC: There are well‐known comic adaptations and horror comics featuring mummies.  
• Connection to EGYPT: Mummies are by far the most iconic symbol of ancient Egypt.  
• Connection to HOOD: A mummy is wrapped in strips of fabric that serve as a sort of hood or covering.  
• Connection to STRING: The long, bandage-like strips used to wrap a mummy are reminiscent of strings.  

Additionally, “Mummy” is a single, simple English word that directly evokes these primary and widely recognized themes. It does not form any inflection or derivation of the target words, nor does it incorporate or hide a board word. Most importantly, it has no connection whatsoever to the assassin word “AGENT.”

Mummy
